DENSE RETRIEVAL

In [ ]:
!pip install -U beir

In [2]:
import pathlib, os
import time
import pandas as pd
import random
import requests
import json
import torch
import torch.multiprocessing as mp
from tqdm.notebook import tqdm
from tqdm.autonotebook import trange
from beir import util, LoggingHandler
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval import models
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES
from beir.datasets.data_loader import GenericDataLoader

In [3]:
def load(dataset):
  hostname = 'localhost'
  index_name = dataset
  url = 'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip'.format(dataset)
  out_dir = os.path.join(os.getcwd(), 'datasets')
  data_path = util.download_and_unzip(url, out_dir)
  return GenericDataLoader(data_path).load(split='test') # 'test', 'train', 'dev'


In [4]:
def retrieve(corpus, queries, qrels, model_name, batch_size, score_function):
  if batch_size is None:
    dr_model = EvaluateRetrieval(DRES(models.SentenceBERT(model_name)), score_function=score_function)
  else:
    dr_model = EvaluateRetrieval(
        DRES(models.SentenceBERT(model_name), batch_size=batch_size), score_function=score_function)
  res_model = dr_model.retrieve(corpus, queries)
  return dr_model.evaluate(qrels, res_model, dr_model.k_values)

In [5]:
def evaluate(corpus, queries, qrels, model_name, batch_size, score_function):
  ndcg, _map, recall, precision = retrieve(corpus, queries, qrels, model_name, batch_size, score_function)
  c_map = 'MAP@10'
  c_map_h = 'MAP@100'
  c_map_t = 'MAP@1000'
  c_ndcg = 'NDCG@10'
  c_ndcg_h = 'NDCG@100'
  c_ndcg_t = 'NDCG@1000'
  c_pre = 'P@10'
  c_pre_h = 'P@100'
  c_pre_t = 'P@1000'
  c_rec = 'Recall@10'
  c_rec_h = 'Recall@100'
  c_rec_t = 'Recall@1000'
  eval_dict = {
      c_map: [_map[c_map]], 
      c_map_h: [_map[c_map_h]],
      c_map_t: [_map[c_map_t]],
      c_ndcg: [ndcg[c_ndcg]], 
      c_ndcg_h: [ndcg[c_ndcg_h]],
      c_ndcg_t: [ndcg[c_ndcg_t]],
      c_pre: [precision[c_pre]], 
      c_pre_h: [precision[c_pre_h]],
      c_pre_t: [precision[c_pre_t]],
      c_rec: [recall[c_rec]],
      c_rec_h: [recall[c_rec_h]],
      c_rec_t: [recall[c_rec_t]]}
  eval_df = pd.DataFrame(data=eval_dict)
  eval_df.index = [model_name]
  return eval_df

In [6]:
fq_corpus, fq_queries, fq_qrels = load('fiqa')
fq_als = evaluate(fq_corpus, fq_queries, fq_qrels, 'allenai-specter', 128, 'dot')
fq_amp = evaluate(fq_corpus, fq_queries, fq_qrels, 'all-mpnet-base-v2', 128, 'dot')
fq_adr = evaluate(fq_corpus, fq_queries, fq_qrels, 'all-distilroberta-v1', 128, 'dot')
fq_aml = evaluate(fq_corpus, fq_queries, fq_qrels, 'all-MiniLM-L12-v2', 128, 'dot')
fq_mmp = evaluate(fq_corpus, fq_queries, fq_qrels, 'multi-qa-mpnet-base-dot-v1', 128, 'dot')
fq_mml = evaluate(fq_corpus, fq_queries, fq_qrels, 'multi-qa-MiniLM-L6-cos-v1', 128, 'cos_sim')
fq_mdd = evaluate(fq_corpus, fq_queries, fq_qrels, 'multi-qa-distilbert-dot-v1', 128, 'dot')
fq_mdc = evaluate(fq_corpus, fq_queries, fq_qrels, 'multi-qa-distilbert-cos-v1', 128, 'cos_sim')
fq_msdd = evaluate(fq_corpus, fq_queries, fq_qrels, 'msmarco-distilbert-base-tas-b', 128, 'dot')
fq_msdc = evaluate(fq_corpus, fq_queries, fq_qrels, 'msmarco-distilbert-dot-v5', 128, 'dot')
fq_msbc = evaluate(fq_corpus, fq_queries, fq_qrels, 'msmarco-bert-co-condensor', 128, 'dot')
fq_msra = evaluate(fq_corpus, fq_queries, fq_qrels, 'msmarco-roberta-base-ance-firstp', None, 'dot')

In [7]:
ag_corpus, ag_queries, ag_qrels = load('arguana')
ag_als = evaluate(ag_corpus, ag_queries, ag_qrels, 'allenai-specter', 128, 'dot')
ag_amp = evaluate(ag_corpus, ag_queries, ag_qrels, 'all-mpnet-base-v2', 128, 'dot')
ag_adr = evaluate(ag_corpus, ag_queries, ag_qrels, 'all-distilroberta-v1', 128, 'dot')
ag_aml = evaluate(ag_corpus, ag_queries, ag_qrels, 'all-MiniLM-L12-v2', 128, 'dot')
ag_mmp = evaluate(ag_corpus, ag_queries, ag_qrels, 'multi-qa-mpnet-base-dot-v1', 128, 'dot')
ag_mml = evaluate(ag_corpus, ag_queries, ag_qrels, 'multi-qa-MiniLM-L6-cos-v1', 128, 'cos_sim')
ag_mdd = evaluate(ag_corpus, ag_queries, ag_qrels, 'multi-qa-distilbert-dot-v1', 128, 'dot')
ag_mdc = evaluate(ag_corpus, ag_queries, ag_qrels, 'multi-qa-distilbert-cos-v1', 128, 'cos_sim')
ag_msdd = evaluate(ag_corpus, ag_queries, ag_qrels, 'msmarco-distilbert-base-tas-b', 128, 'dot')
ag_msdc = evaluate(ag_corpus, ag_queries, ag_qrels, 'msmarco-distilbert-dot-v5', 128, 'dot')
ag_msbc = evaluate(ag_corpus, ag_queries, ag_qrels, 'msmarco-bert-co-condensor', 128, 'dot')
ag_msra = evaluate(ag_corpus, ag_queries, ag_qrels, 'msmarco-roberta-base-ance-firstp', None, 'dot')

In [ ]:
tu_corpus, tu_queries, tu_qrels = load('webis-touche2020')
tu_als = evaluate(tu_corpus, tu_queries, tu_qrels, 'allenai-specter', 128, 'dot')
tu_amp = evaluate(tu_corpus, tu_queries, tu_qrels, 'all-mpnet-base-v2', 128, 'dot')
tu_adr = evaluate(tu_corpus, tu_queries, tu_qrels, 'all-distilroberta-v1', 128, 'dot')
tu_aml = evaluate(tu_corpus, tu_queries, tu_qrels, 'all-MiniLM-L12-v2', 128, 'dot')
tu_mmp = evaluate(tu_corpus, tu_queries, tu_qrels, 'multi-qa-mpnet-base-dot-v1', 128, 'dot')
tu_mml = evaluate(tu_corpus, tu_queries, tu_qrels, 'multi-qa-MiniLM-L6-cos-v1', 128, 'cos_sim')
tu_mdd = evaluate(tu_corpus, tu_queries, tu_qrels, 'multi-qa-distilbert-dot-v1', 128, 'dot')
tu_mdc = evaluate(tu_corpus, tu_queries, tu_qrels, 'multi-qa-distilbert-cos-v1', 128, 'cos_sim')
tu_msdd = evaluate(tu_corpus, tu_queries, tu_qrels, 'msmarco-distilbert-base-tas-b', 128, 'dot')
tu_msdc = evaluate(tu_corpus, tu_queries, tu_qrels, 'msmarco-distilbert-dot-v5', 128, 'dot')
tu_msbc = evaluate(tu_corpus, tu_queries, tu_qrels, 'msmarco-bert-co-condensor', 128, 'dot')
tu_msra = evaluate(tu_corpus, tu_queries, tu_qrels, 'msmarco-roberta-base-ance-firstp', None, 'dot')

In [9]:
qa_corpus, qa_queries, qa_qrels = load('quora')
qa_als = evaluate(qa_corpus, qa_queries, qa_qrels, 'allenai-specter', 128, 'dot')
qa_amp = evaluate(qa_corpus, qa_queries, qa_qrels, 'all-mpnet-base-v2', 128, 'dot')
qa_adr = evaluate(qa_corpus, qa_queries, qa_qrels, 'all-distilroberta-v1', 128, 'dot')
qa_aml = evaluate(qa_corpus, qa_queries, qa_qrels, 'all-MiniLM-L12-v2', 128, 'dot')
qa_mmp = evaluate(qa_corpus, qa_queries, qa_qrels, 'multi-qa-mpnet-base-dot-v1', 128, 'dot')
qa_mml = evaluate(qa_corpus, qa_queries, qa_qrels, 'multi-qa-MiniLM-L6-cos-v1', 128, 'cos_sim')
qa_mdd = evaluate(qa_corpus, qa_queries, qa_qrels, 'multi-qa-distilbert-dot-v1', 128, 'dot')
qa_mdc = evaluate(qa_corpus, qa_queries, qa_qrels, 'multi-qa-distilbert-cos-v1', 128, 'cos_sim')
qa_msdd = evaluate(qa_corpus, qa_queries, qa_qrels, 'msmarco-distilbert-base-tas-b', 128, 'dot')
qa_msdc = evaluate(qa_corpus, qa_queries, qa_qrels, 'msmarco-distilbert-dot-v5', 128, 'dot')
qa_msbc = evaluate(qa_corpus, qa_queries, qa_qrels, 'msmarco-bert-co-condensor', 128, 'dot')
qa_msra = evaluate(qa_corpus, qa_queries, qa_qrels, 'msmarco-roberta-base-ance-firstp', None, 'dot')



---



**FIQA** - [Homepage](https://sites.google.com/view/fiqa/home) - [Paper](https://www.researchgate.net/publication/324629350_WWW'18_Open_Challenge_Financial_Opinion_Mining_and_Question_Answering)

In [10]:
pd.concat([fq_als, fq_amp, fq_adr, fq_aml, fq_mmp, fq_mml, fq_mdd, fq_mdc, fq_msdd, fq_msdc, fq_msbc, fq_msra])

,MAP@10,MAP@100,MAP@1000,NDCG@10,NDCG@100,NDCG@1000,P@10,P@100,P@1000,Recall@10,Recall@100,Recall@1000
allenai-specter,0.03782,0.04428,0.04583,0.06148,0.09875,0.14101,0.02083,0.00574,0.00130,0.08554,0.23704,0.50673
all-mpnet-base-v2,0.41117,0.43167,0.43323,0.49963,0.56564,0.58932,0.13889,0.02091,0.00251,0.58112,0.81969,0.96187
all-distilroberta-v1,0.31174,0.33246,0.33442,0.39425,0.46596,0.49713,0.11373,0.01869,0.00241,0.47536,0.73675,0.92236
all-MiniLM-L12-v2,0.29694,0.31593,0.31795,0.37274,0.44379,0.47803,0.10571,0.01779,0.00237,0.43460,0.70017,0.90838
multi-qa-mpnet-base-dot-v1,0.40186,0.42360,0.42518,0.48786,0.55690,0.58037,0.13256,0.02048,0.00249,0.56257,0.80788,0.94560
multi-qa-MiniLM-L6-cos-v1,0.28995,0.30868,0.31045,0.36346,0.43275,0.46577,0.10231,0.01728,0.00231,0.43169,0.68812,0.89008
multi-qa-distilbert-dot-v1,0.34930,0.36924,0.37113,0.43201,0.49948,0.52885,0.11929,0.01881,0.00239,0.50730,0.75268,0.92694
multi-qa-distilbert-cos-v1,0.32248,0.34045,0.34245,0.40090,0.46721,0.49872,0.11173,0.01799,0.00236,0.47715,0.72436,0.90863
msmarco-distilbert-base-tas-b,0.23319,0.24772,0.24961,0.30024,0.36041,0.39641,0.08117,0.01434,0.00208,0.36756,0.59293,0.80835
msmarco-distilbert-dot-v5,0.22339,0.23781,0.23975,0.28607,0.34700,0.38421,0.07855,0.01380,0.00204,0.34047,0.57077,0.79600




---



**ARGUANA** - [Homepage](http://argumentation.bplaced.net/arguana/data) - [Paper](https://aclanthology.org/P18-1023.pdf)

In [11]:
pd.concat([ag_als, ag_amp, ag_adr, ag_aml, ag_mmp, ag_mml, ag_mdd, ag_mdc, ag_msdd, ag_msdc, ag_msbc, ag_msra])

,MAP@10,MAP@100,MAP@1000,NDCG@10,NDCG@100,NDCG@1000,P@10,P@100,P@1000,Recall@10,Recall@100,Recall@1000
allenai-specter,0.25379,0.26748,0.26795,0.32007,0.38744,0.40196,0.05341,0.00851,0.00097,0.53414,0.85135,0.97013
all-mpnet-base-v2,0.37692,0.38986,0.38991,0.46521,0.51954,0.52087,0.07511,0.00986,0.00100,0.75107,0.98649,0.99644
all-distilroberta-v1,0.38394,0.39514,0.39520,0.47935,0.52573,0.52711,0.07866,0.00986,0.00100,0.78663,0.98578,0.99644
all-MiniLM-L12-v2,0.38083,0.39264,0.39273,0.47128,0.52093,0.52316,0.07632,0.00979,0.00100,0.76316,0.97866,0.99573
multi-qa-mpnet-base-dot-v1,0.41450,0.42569,0.42573,0.50390,0.54996,0.55097,0.07916,0.00989,0.00100,0.79161,0.98862,0.99644
multi-qa-MiniLM-L6-cos-v1,0.40196,0.41336,0.41343,0.49110,0.53828,0.53993,0.07788,0.00982,0.00100,0.77881,0.98222,0.99502
multi-qa-distilbert-dot-v1,0.32681,0.34056,0.34075,0.41017,0.47188,0.47663,0.06799,0.00957,0.00099,0.67994,0.95661,0.99360
multi-qa-distilbert-cos-v1,0.41881,0.42913,0.42920,0.51080,0.55215,0.55373,0.08080,0.00982,0.00099,0.80797,0.98222,0.99431
msmarco-distilbert-base-tas-b,0.33946,0.35126,0.35159,0.42700,0.47890,0.48601,0.07112,0.00942,0.00099,0.71124,0.94168,0.99431
msmarco-distilbert-dot-v5,0.26949,0.28396,0.28433,0.34886,0.41464,0.42493,0.06081,0.00906,0.00099,0.60811,0.90612,0.98720




---



**TOUCHÉ** - [Homepage](https://webis.de/events/touche-20/) - [Paper](https://www.researchgate.net/publication/344371868_Overview_of_Touche_2020_Argument_Retrieval)

In [12]:
pd.concat([tu_als, tu_amp, tu_adr, tu_aml, tu_mmp, tu_mml, tu_mdd, tu_mdc, tu_msdd, tu_msdc, tu_msbc, tu_msra])

,MAP@10,MAP@100,MAP@1000,NDCG@10,NDCG@100,NDCG@1000,P@10,P@100,P@1000,Recall@10,Recall@100,Recall@1000
allenai-specter,0.03532,0.06169,0.07262,0.10216,0.18432,0.30415,0.09796,0.04265,0.01200,0.07288,0.27132,0.64884
all-mpnet-base-v2,0.07557,0.12238,0.13773,0.19934,0.30526,0.43148,0.17551,0.06408,0.01463,0.13002,0.40980,0.79409
all-distilroberta-v1,0.08344,0.13906,0.15382,0.19502,0.32623,0.44518,0.17551,0.07061,0.01494,0.13723,0.45072,0.80799
all-MiniLM-L12-v2,0.06632,0.11485,0.13031,0.17224,0.29288,0.41613,0.15714,0.06408,0.01451,0.12006,0.41017,0.78632
multi-qa-mpnet-base-dot-v1,0.07881,0.12627,0.14245,0.19835,0.30955,0.43443,0.17959,0.06694,0.01486,0.13972,0.42313,0.80003
multi-qa-MiniLM-L6-cos-v1,0.06488,0.11654,0.13230,0.15608,0.28808,0.41603,0.15102,0.06490,0.01488,0.12267,0.41106,0.80569
multi-qa-distilbert-dot-v1,0.09754,0.14994,0.16487,0.23677,0.33935,0.45776,0.22041,0.07204,0.01494,0.16087,0.44583,0.79788
multi-qa-distilbert-cos-v1,0.08505,0.13406,0.14946,0.19614,0.30961,0.43464,0.19184,0.06776,0.01494,0.14859,0.42850,0.80636
msmarco-distilbert-base-tas-b,0.06875,0.12285,0.13922,0.16249,0.30565,0.42964,0.14694,0.06755,0.01498,0.12214,0.43100,0.81051
msmarco-distilbert-dot-v5,0.08696,0.14169,0.15727,0.22404,0.33113,0.45111,0.21224,0.07041,0.01484,0.15157,0.42629,0.79315




---



**QUORA** - [Homepage](https://quoradata.quora.com/First-Quora-Dataset-Release-Question-Pairs) - [Paper](https://arxiv.org/abs/1907.01041)

In [13]:
pd.concat([qa_als, qa_amp, qa_adr, qa_aml, qa_mmp, qa_mml, qa_mdd, qa_mdc, qa_msdd, qa_msdc, qa_msbc, qa_msra])

,MAP@10,MAP@100,MAP@1000,NDCG@10,NDCG@100,NDCG@1000,P@10,P@100,P@1000,Recall@10,Recall@100,Recall@1000
allenai-specter,0.59299,0.60174,0.60235,0.63977,0.67275,0.68401,0.09753,0.01257,0.00143,0.73069,0.86971,0.94583
all-mpnet-base-v2,0.83546,0.84229,0.84243,0.87455,0.88767,0.88857,0.13418,0.01536,0.00157,0.95032,0.99559,0.99987
all-distilroberta-v1,0.82683,0.83383,0.83397,0.86786,0.88145,0.88234,0.13383,0.01537,0.00157,0.94876,0.99570,0.99990
all-MiniLM-L12-v2,0.83847,0.84491,0.84506,0.87761,0.88981,0.89081,0.13452,0.01532,0.00157,0.95329,0.99520,0.99991
multi-qa-mpnet-base-dot-v1,0.83533,0.84197,0.84212,0.87446,0.88747,0.88836,0.13401,0.01536,0.00157,0.95108,0.99587,0.99990
multi-qa-MiniLM-L6-cos-v1,0.83294,0.83961,0.83980,0.87211,0.88551,0.88685,0.13291,0.01523,0.00156,0.94653,0.99297,0.99951
multi-qa-distilbert-dot-v1,0.84981,0.85599,0.85613,0.88806,0.89874,0.89948,0.13583,0.01536,0.00157,0.96163,0.99694,0.99993
multi-qa-distilbert-cos-v1,0.84124,0.84781,0.84796,0.87969,0.89212,0.89307,0.13425,0.01534,0.00157,0.95307,0.99560,0.99984
msmarco-distilbert-base-tas-b,0.79191,0.79929,0.79956,0.83509,0.85353,0.85590,0.12644,0.01489,0.00156,0.91876,0.98620,0.99840
msmarco-distilbert-dot-v5,0.78896,0.79630,0.79657,0.83266,0.85117,0.85380,0.12573,0.01479,0.00155,0.91665,0.98418,0.99781


end of fun.